### Import libraries

In [1]:
import numpy as np
import pandas as pd
from absl import flags
import tensorflow as tf
import tensorflow_privacy
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
%matplotlib inline
tf.get_logger().setLevel('ERROR')
tf.compat.v1.disable_v2_behavior()

### Load Dataset

In [2]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 10)
testing: (10000, 32, 32, 3) (10000, 10)


### Model3: Custom CNN

In [3]:
# Hyper-parameters
epochs = 40
batch_size = 1250
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25
delta = 1e-5

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')
    
# Compute RDP
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders=orders)
# Calculate epsilon
epsilon = get_privacy_spent(orders, rdp, target_delta=delta)[0]
epsilon

4.413400644320362

In [4]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 5, strides=1, padding='same', activation='relu', input_shape=(32, 32, 3)),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, 5, strides=1, padding='valid', activation='relu'),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, 3, strides=2, padding='valid', activation='relu'),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.ReLU(max_value = 50, negative_slope = 1.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        4864      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        51232     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 32)          9248      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 32)         0

In [7]:
# define optimizer (dp-sgd) 
# optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
#     l2_norm_clip=l2_norm_clip,
#     noise_multiplier=noise_multiplier,
#     num_microbatches=num_microbatches,
#     learning_rate=learning_rate)

# define optimizer (dp-adam)
optimizer = tensorflow_privacy.DPKerasAdamOptimizer(
    l2_norm_clip, noise_multiplier, num_microbatches)

# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [8]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Train on 50000 samples, validate on 10000 samples
Metal device set to: Apple M1 Pro


2022-03-20 11:34:44.555952: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-20 11:34:44.556077: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-20 11:34:45.010257: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-20 11:34:45.019226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-20 11:34:45.528431: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-20 11:34:45.687538: I tensorflow/core/grappler/o

Epoch 1/40


2022-03-20 11:34:45.784209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1250/50000 [..............................] - ETA: 50:25 - loss: 2.3005 - acc: 0.1160

KeyboardInterrupt: 

### Model1: Simple CNN

In [11]:
# Hyper-parameters
epochs = 20
batch_size = 500

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [12]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [13]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [14]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples


2022-03-08 16:42:30.484258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.402756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/20


2022-03-08 16:42:31.686466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 16:42:31.846137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.2188 - acc: 0.1664 

2022-03-08 17:04:09.816648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 1299s 26ms/sample - loss: 2.2188 - acc: 0.1664 - val_loss: 2.1000 - val_acc: 0.2320
Epoch 2/20
50000/50000 [==============================] - 1223s 24ms/sample - loss: 2.0589 - acc: 0.2534 - val_loss: 1.9255 - val_acc: 0.3094
Epoch 3/20
50000/50000 [==============================] - 1212s 24ms/sample - loss: 1.9223 - acc: 0.3040 - val_loss: 1.8572 - val_acc: 0.3305
Epoch 4/20
50000/50000 [==============================] - 1217s 24ms/sample - loss: 1.8465 - acc: 0.3341 - val_loss: 1.8674 - val_acc: 0.3170
Epoch 5/20
50000/50000 [==============================] - 1208s 24ms/sample - loss: 1.7838 - acc: 0.3603 - val_loss: 1.7684 - val_acc: 0.3776
Epoch 6/20
50000/50000 [==============================] - 1221s 24ms/sample - loss: 1.7363 - acc: 0.3805 - val_loss: 1.7594 - val_acc: 0.3869
Epoch 7/20
50000/50000 [==============================] - 1227s 25ms/sample - loss: 1.6895 - acc: 0.4000 - val_loss: 1.7437 - val_acc: 0.3961
Epoch 8/20
50000/

In [15]:
# Calculate privacy performance
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=x_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

DP-SGD with sampling rate = 1% and noise_multiplier = 1.3 iterated over 2000 steps satisfies differential privacy with eps = 1.8 and delta = 1e-05.
The optimal RDP order is 11.0.


(1.798929515280597, 11.0)

### Model2: VGG with transfer learning

In [3]:
# Hyper-parameters
epochs = 50
batch_size = 1250

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 250
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [4]:
#Building a VGG model with pretrained weights

vgg_model=tf.keras.Sequential()

#https://keras.io/api/applications/vgg/#vgg16-function
#https://keras.io/guides/transfer_learning/
#https://chroniclesofai.com/transfer-learning-with-keras-resnet-50/
pre_vgg_model=tf.keras.applications.VGG16(include_top=False,weights="imagenet",
                                          classes=10,pooling=max,input_shape=(32,32,3))
pre_vgg_model.trainable=False
vgg_model.add(pre_vgg_model)
vgg_model.add(tf.keras.layers.Flatten())
vgg_model.add(tf.keras.layers.Dense(512, activation='tanh'))
vgg_model.add(tf.keras.layers.Dense(10, activation='softmax'))


Metal device set to: Apple M1 Pro


2022-03-14 00:07:48.042660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-14 00:07:48.042815: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-14 00:07:48.050509: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-14 00:07:48.050609: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:07:48.065426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:07:48.153230: I tensorflow/core/grappler/o

In [5]:
vgg_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# define optimizer (dp-sgd) 
vgg_optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
vgg_loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [7]:
# Compile model
vgg_model.compile(optimizer=vgg_optimizer, loss=vgg_loss, metrics=['accuracy'])
# Train model
vgg_model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Train on 50000 samples, validate on 10000 samples


2022-03-14 00:08:06.080186: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50


2022-03-14 00:08:06.388985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:08:06.466079: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-14 00:08:06.528768: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - ETA: 0s - loss: 2.0519 - acc: 0.2842

/Users/Mahos/miniforge3/envs/shm_tf/lib/python3.9/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-03-14 00:10:55.361394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50000/50000 [==============================] - 174s 3ms/sample - loss: 2.0519 - acc: 0.2842 - val_loss: 1.8509 - val_acc: 0.3751
Epoch 2/50
50000/50000 [==============================] - 158s 3ms/sample - loss: 1.7477 - acc: 0.4120 - val_loss: 1.6785 - val_acc: 0.4330
Epoch 3/50
50000/50000 [==============================] - 157s 3ms/sample - loss: 1.6229 - acc: 0.4513 - val_loss: 1.5985 - val_acc: 0.4520
Epoch 4/50
50000/50000 [==============================] - 158s 3ms/sample - loss: 1.5563 - acc: 0.4692 - val_loss: 1.5471 - val_acc: 0.4685
Epoch 5/50
50000/50000 [==============================] - 156s 3ms/sample - loss: 1.5081 - acc: 0.4849 - val_loss: 1.5045 - val_acc: 0.4841
Epoch 6/50
50000/50000 [==============================] - 158s 3ms/sample - loss: 1.4763 - acc: 0.4935 - val_loss: 1.4800 - val_acc: 0.4906
Epoch 7/50
50000/50000 [==============================] - 160s 3ms/sample - loss: 1.4505 - acc: 0.5006 - val_loss: 1.4585 - val_acc: 0.4972
Epoch 8/50
50000/50000 [=======

In [8]:
# noise_multiplier = FLAGS.noise_multiplier
# sampling_probability = FLAGS.batch_size / 60000
# steps = FLAGS.epochs * 60000 // FLAGS.batch_size

In [9]:
# from absl import app
# from absl import flags
# from absl import logging
# FLAGS = flags.FLAGS
orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
rdp = compute_rdp(q = batch_size / 50000,
                  noise_multiplier = noise_multiplier,
                  steps = epochs * 50000 // batch_size,
                  orders=orders)

In [10]:
epsilon = get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
epsilon

4.980956406486334

### Model3: HOG classification